In [1]:
from keras.datasets import cifar100
from keras.callbacks import EarlyStopping
from UniversalStyleTransfer import UniversalStyleTransfer
import numpy as np

/home/tadenoud/.virtualenvs/ml/local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load the dataset

In [2]:
epochs = 200
batch_size = 100
w, h, c = 64, 64, 3

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        '../datasets/tiny-imagenet-200/train',
        target_size=(w, h),
        batch_size=batch_size,
        class_mode='input')

test_datagen = ImageDataGenerator(
        rescale=1./255)

val_generator = test_datagen.flow_from_directory(
        '../datasets/tiny-imagenet-200/val',
        target_size=(w, h),
        batch_size=batch_size,
        class_mode='input')

def gen(generator):
    """Yield the labels twice to allow multiple loss functions."""
    while True:
        x, y = generator.next()
        yield x, [y, y]
        
train_generator = gen(train_generator)
val_generator = gen(val_generator)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 1 classes.


In [5]:
model = UniversalStyleTransfer((w, h, c), lam=0.1)

# TODO: Improve API so users don't need to manually specify loss functions
model.compile(optimizer='adam', loss={
    'reconstruction': model.reconstruction_loss,
    'feature': model.feature_loss
})

Compiling autoencoder 1


In [ ]:
# TODO: Improve API so users don't need to pass two label datasets
train = True

if train:    
    model.fit_generator(
        train_generator,
        epochs=epochs,
        steps_per_epoch=100000 / batch_size,
        callbacks=[
            EarlyStopping(patience=3, min_delta=0.1)
        ],
        validation_data=val_generator,
        validation_steps=10000 / batch_size,
    )

    model.save_weights(prefix="autoencoder")

Training autoencoder 1
Epoch 1/200
1563/1562 [==============================] - 137s 88ms/step - loss: 4348.5509 - reconstruction_loss: 1645.6882 - feature_loss: 2702.8626 - val_loss: 3860.9689 - val_reconstruction_loss: 1382.8622 - val_feature_loss: 2478.1067
Epoch 2/200
1563/1562 [==============================] - 136s 87ms/step - loss: 3842.7134 - reconstruction_loss: 1375.9073 - feature_loss: 2466.8062 - val_loss: 3859.9575 - val_reconstruction_loss: 1377.5677 - val_feature_loss: 2482.3898
Epoch 3/200
1563/1562 [==============================] - 137s 87ms/step - loss: 3835.6413 - reconstruction_loss: 1372.8548 - feature_loss: 2462.7865 - val_loss: 3845.0540 - val_reconstruction_loss: 1379.4727 - val_feature_loss: 2465.5812
Epoch 4/200
1563/1562 [==============================] - 110s 70ms/step - loss: 3826.5033 - reconstruction_loss: 1369.5260 - feature_loss: 2456.9773 - val_loss: 3852.4321 - val_reconstruction_loss: 1380.8588 - val_feature_loss: 2471.5733 - loss: 3829.3732 - recon

In [ ]:
model.load_weights(prefix="../weights/autoencoder")

## Sanity check autoencoder training

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# Sanity check autoencoder results
img, _ = train_generator.next()
img = img[0]
plt.imshow(img)
plt.show()

for autoencoder in model._autoencoders:
    pred = autoencoder.predict(np.expand_dims(img, axis=0))

    plt.imshow(pred[0].squeeze())
    plt.show()


## Test style transfer

In [ ]:
img = x_train[0]
style = x_train[5]

plt.imshow(img)
plt.show()
plt.imshow(style)
plt.show()

# show result
pred = model.stylize(img, style, 0.)
plt.imshow(pred)
plt.show()